# Implementação Concisa de Regressão Linear

Mostraremos como implementar
o modelo de regressão linear
de forma concisa, usando APIs de alto nível

In [2]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

Chamar a API existente em uma estrutura para ler os dados

In [5]:
def load_array(data_arrays, batch_size, is_train=True):  
    """Construct a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[ 0.1268, -0.8247],
         [-1.8586,  0.2485],
         [ 1.7878, -0.1247],
         [-0.3924, -1.2406],
         [ 1.1753, -1.0299],
         [ 0.2604,  0.7576],
         [ 0.1032,  0.3114],
         [ 1.0892, -0.1115],
         [ 0.9290, -1.0566],
         [ 2.0271,  1.8531]]),
 tensor([[ 7.2536],
         [-0.3701],
         [ 8.2033],
         [ 7.6378],
         [10.0588],
         [ 2.1468],
         [ 3.3497],
         [ 6.7665],
         [ 9.6711],
         [ 1.9447]])]

Usar as camadas predefinidas de uma estrutura

In [6]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

Inicializar os parâmetros do modelo

In [7]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

A classe `MSELoss` calcula o erro quadrático médio, também conhecido como norma $ L_2 $ quadrada

In [8]:
loss = nn.MSELoss()

Instanciamos uma instância `SGD`

In [9]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

O loop de treinamento em si é surpreendentemente semelhante
ao que fizemos ao implementar tudo do zero

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000299
epoch 2, loss 0.000103
epoch 3, loss 0.000103


Comparamos os parâmetros do modelo aprendidos pelo treinamento em dados finitos
e os parâmetros reais

In [11]:
w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([2.6226e-05, 2.5201e-04])
error in estimating b: tensor([-0.0008])
